# Forel Ule

_Work in progress_

In [ ]:
from dask.distributed import Client

client = Client("tcp://10.0.103.154:40209")
client.restart()
client

In [ ]:
# Data tools
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime

# Datacube
import datacube
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from odc.algo import xr_reproject   # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_warp.py
from datacube.utils.geometry import GeoBox, box  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/geometry/_base.py
from datacube.utils.rio import configure_s3_access

# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
from holoviews.operation.datashader import rasterize
hv.extension("bokeh", logo=False)

# Python
import sys, os, re
import scipy.spatial.distance
import scipy.interpolate as interpolate
from scipy.interpolate import interp1d
import math

# Optional EASI tools
sys.path.append(os.path.expanduser('~/hub-notebooks/scripts'))
import notebook_utils

## Load data

In [ ]:
#read in Forel Ule Index FUME file
csv = os.path.expanduser('~/data/WQ/FUI_ATAN210.csv')
fui = pd.read_csv(sep = "\t", filepath_or_buffer=csv, names = ["value", "atan"])

In [ ]:
# Connect to datacube database
dc = datacube.Datacube()

In [ ]:
# Define temporal and spatial extents to pass to `dc.load` for the data extraction:
#Lake Burley Griffin
#query = {'lat': (-35.2803, -35.3147),
#         'lon': (149.0627, 149.157),
#         'time':('2018-10-15', '2018-12-31'),
#         'crs': 'EPSG:4326'}
#Lake Tuggeranong
# query = {'lat': (-35.397, -35.429),
#         'lon': (149.065, 149.085),
#         'time':('2018-12-01', '2019-03-14'),
#         'crs': 'EPSG:4326'}
#Lake Hume
#query = {'lat': (-35.947, -36.27),
#         'lon': (147.000, 147.366),
#         'time':('2018-12-01', '2019-03-14'),
#         'crs': 'EPSG:4326'}
#Melbourne Treatment Ponds
#query = {'lat': (-37.972, -38.024),
#        'lon': (144.558, 144.709),
#        'time':('2018-08-01', '2019-03-14'),
#        'crs': 'EPSG:4326'}
#Burdekin
# query = {'lat': (-19.21, -19.88),
#         'lon': (147.54, 147.99),
#         'time':('2019-02-10', '2019-06-14'),
#         'crs': 'EPSG:4326'}
#Leo Wetlands
#query = {'lat': (-35.9075, -35.968),
#         'lon': (144.9033, 145.009),
#        'time':('2016-12-11', '2017-01-31'),
#         'crs': 'EPSG:4326'}
#Menindee
#query = {'lat': (-32.2305, -32.3179),
#          'lon': (142.3945, 142.5809),
#          'time':('2018-10-15', '2019-02-10'),
#          'crs': 'EPSG:4326'}
#specify bands if needed
#bands = ['coastal_aerosol', 'blue', 'green', 'red']
# Vietnam - Rach Gia
# query = {'lat': (9.5, 10.5),
#         'lon': (105.5, 106.5),
#         'time':('2019-02-10', '2020-06-14'),
#         'crs': 'EPSG:4326'}
# Vietnam - Ha Long (small)
# query = {'lat': (20.7, 21.1), 
#         'lon': (106.7, 107.7),
#         'time':('2019-01-01', '2021-01-31'),
#         }
# Vietnam - Ha Long (large)
# query = {'lat': (17.5, 23.0), 
#         'lon': (103.5, 110.0),
#         'time':('2019-01-01', '2021-01-31'),
#         }
# Vietnam - Ha Long (small)
query = {'lat': (20.9, 21.1), 
        'lon': (106.8, 107.3),
        'time':('2019-02-01', '2021-01-31'),
        }

In [ ]:
# Get data in native projection (UTM)

product = 'usgs_espa_ls8c1_sr'
measurements = ['red', 'green', 'blue', 'coastal_aerosol', 'swir1', 'nir',  'pixel_qa']

# product = 'cophub_s2_sr_sen2cor'
# measurements = ['coastal_aerosol', 'blue', 'green', 'red', 'red_edge_1']

query.update({
    'product': product,                     # Product name
    'measurements': measurements,
})

# Most common CRS
native_crs = notebook_utils.mostcommon_crs(dc, query)
print(f'Native CRS: {native_crs}')

# query.update({
#     'output_crs': native_crs,               # EPSG code
#     'resolution': (-30, 30),                # Target resolution
#     'group_by': 'solar_day',                # Scene ordering
#     'dask_chunks': {'x': 2048, 'y': 2048},  # Dask chunks
# })

query.update({
    'output_crs': 'epsg:4326',               # EPSG code
    'resolution': (-0.003, 0.003),                # Target resolution
    'group_by': 'solar_day',                # Scene ordering
    'dask_chunks': {'longitude': 2048, 'latitude': 2048},  # Dask chunks
})

data = dc.load(**query)

notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

# Calculate valid (not nodata) masks for each layer
valid_mask = masking.valid_data_mask(data)

## Scaling and masking

In [ ]:
# Add cloud masking

good_pixel_flags = {
    'snow': 'no_snow',                    # 'no_snow', 'snow'
#     'clear': 'clear_land',                # 'no_clear_land', 'clear_land'
    'cloud': 'no_cloud',                  # 'no_cloud', 'cloud'
#     'water': 'water',                     # 'no_water', 'water'
    'nodata': False,                      # False, True
    'cloud_shadow': 'no_cloud_shadow',    # 'no_cloud_shadow', 'cloud_shadow'
#     'cloud_confidence': 'medium',         # 'none', 'low', 'medium', 'high'
#     'cirrus_confidence': 'medium',        # 'none', 'low', 'medium', 'high'
#     'terrain_occlusion': 'no_occlusion',  # 'no_occlusion', 'occlusion'
}

flag_name = 'pixel_qa'
good_pixel_mask = masking.make_mask(data[flag_name], **good_pixel_flags)
# usgs_espa_ls8c1_sr, usgs_espa_ls8c1_ar
scale = 0.0001
offset = 0.
layer = data.where(valid_mask & good_pixel_mask) * scale + offset

In [ ]:
#convert to remote sensing sub-surface reflectance (surface refl / pi -> lambertian sub-surface)
data = layer/math.pi

# #mask land using MNDWI
mndwi = (data.green - data.swir1) / (data.green + data.swir1)
    
data = data.where(mndwi > 0)
# data = data.where(data.nir < 0.07)

In [ ]:
# Remove time layers that are full of nulls

test = data.dropna('time', how='all')

In [ ]:
notebook_utils.heading(notebook_utils.xarray_object_size(test))
display(test)
data = test

## Forel Ule

In [ ]:
# Approximation of tristimulus values from Van der Woerd and Wernard and Lehmann 
# Landsat

if 'ls' in query['product']:
    data['Xtris'] = 11.053*data.coastal_aerosol + 6.950*data.blue + 51.135*data.green + 34.457*data.red
    data['Ytris'] = 1.32*data.coastal_aerosol + 21.053*data.blue + 66.023*data.green + 18.034*data.red
    data['Ztris'] = 58.038*data.coastal_aerosol + 34.931*data.blue + 2.606*data.green + 0.016*data.red
    
# elif 's2' in query['product']:
#     data['Xtris'] = 11.756*Rrs_S2[,1] + 6.423*Rrs_S2[,2] + 53.696*Rrs_S2[,3] + 32.028*Rrs_S2[,4] + 0.529*Rrs_S2[,5]
#     data['Ytris'] = 1.744*Rrs_S2[,1] + 22.289*Rrs_S2[,2] + 65.702*Rrs_S2[,3] + 16.808*Rrs_S2[,4] + 0.192*Rrs_S2[,5]
#     data['Ztris'] = 62.696*Rrs_S2[,1] + 31.101*Rrs_S2[,2] + 1.778*Rrs_S2[,3] + 0.015*Rrs_S2[,4] + 0.000*Rrs_S2[,5]

# Sentinel-2
# pending....

In [ ]:
# Calculate Chromacity coordinates

data['cx']=data.Xtris/(data.Xtris+data.Ytris+data.Ztris)
data['cy']=data.Ytris/(data.Xtris+data.Ytris+data.Ztris)
data['cz']=data.Ztris/(data.Xtris+data.Ytris+data.Ztris)

# Adjust to whitepoint
data['cwx'] = data.cx-(1/3)
data['cwy'] = data.cy-(1/3)

In [ ]:
# **HueAngle**______ calculate atan2 **
landsat_ds = data
landsat_ds['a_i'] = np.arctan2(landsat_ds.cwy, landsat_ds.cwx) * 180 / math.pi
landsat_ds['a_i'] = xr.where(landsat_ds.a_i<0,landsat_ds.a_i+360,landsat_ds.a_i)
landsat_ds.a_i.max()

In [ ]:
def lookup_func(a):
    interp_func = interpolate.interp1d(x=fui.atan, y=fui.value)
    return xr.apply_ufunc(interp_func, a, dask='allowed')

In [ ]:
#Create interpolated Forel layer for FUI atan lookup table 

landsat_ds['forel'] = xr.where(
    landsat_ds.a_i >= fui["atan"][0], 1.0,
    xr.where(
        landsat_ds.a_i <= fui["atan"][200],
        21.0,
        landsat_ds.a_i
    )
)

landsat_ds['forel'] = xr.where(
    landsat_ds.a_i == landsat_ds.forel,
    lookup_func(
        landsat_ds.forel.where(
            landsat_ds.a_i == landsat_ds.forel
        )
    ), landsat_ds.forel
)

#Check the xarray dataset
landsat_ds

In [ ]:
#Round to nearest FU category 
landsat_ds['FUi']=np.round(landsat_ds.forel)
landsat_ds['FUint']=landsat_ds.FUi.astype(int)
landsat_ds['FUint']=xr.where(landsat_ds.FUint<0,0,landsat_ds.FUint)
landsat_ds['FU']=landsat_ds.FUint.where(landsat_ds.forel>0)

#Plot all the time slices
# landsat_ds.FU.plot(col='time',col_wrap=2,robust=True)
#landsat_ds.FU.plot()

## Reproject (from UTM to 4326)

In [ ]:
layer_name = 'FU'
layer = landsat_ds[[layer_name]].persist()
layer.latitude.data

In [ ]:
layer

In [ ]:
target_crs = 'epsg:4326'
target_res = (0.0003, 0.0003)   # approx. 20 m

min_longitude, min_latitude, max_longitude, max_latitude = query['lon'][0], query['lat'][0], query['lon'][1], query['lat'][1]
print(min_longitude, min_latitude, max_longitude, max_latitude)

target = GeoBox.from_geopolygon(box(min_longitude, min_latitude, max_longitude, max_latitude, target_crs), target_res)
layer_geo = xr_reproject(layer, target).persist()
layer_geo

## Generate a plot

In [ ]:
options = {
    'title': f'{query["product"]}: {layer_name}',
    'width': 500,
    'height': 450,
    'aspect': 'equal',
    'cmap': cc.rainbow,
    'clim': (1, 21),                          # Limit the color range depending on the layer_name
#     'clim': (0, 0.5),                          # Limit the color range depending on the layer_name
    'colorbar': True,
    'tools': ['hover'],
}

In [ ]:
# Plot native projection data

# Set the Dataset CRS
# plot_crs = native_crs
plot_crs = 'epsg:4326'
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used
    
layer_plot = layer.hvplot.image(
    x = 'longitude', y = 'latitude',                        # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
    crs = plot_crs,                          # Dataset crs
    projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
    coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:
## Plot 4326 projected data

# Set the Dataset CRS
plot_crs = target_crs
if plot_crs == 'epsg:4326':
    plot_crs = ccrs.PlateCarree()


# Native data and coastline overlay:
# - Comment `crs`, `projection`, `coastline` to plot in native_crs coords
# TODO: Update the axis labels to 'longitude', 'latitude' if `coastline` is used
    
layer_plot = layer_geo.hvplot.image(
    x = 'longitude', y = 'latitude',         # Dataset x,y dimension names
    rasterize = True,                        # Use Datashader
    aggregator = reductions.mean(),          # Datashader selects mean value
    precompute = True,                       # Datashader precomputes what it can
#     crs = plot_crs,                          # Dataset crs
#     projection = ccrs.PlateCarree(),         # Output projection (use ccrs.PlateCarree() when coastline=True)
#     coastline='10m',                         # Coastline = '10m'/'50m'/'110m'
).options(opts.Image(**options)).hist(bin_range = options['clim'])

# display(layer_plot)
# Optional: Change the default time slider to a dropdown list, https://stackoverflow.com/a/54912917
fig = pn.panel(layer_plot, widgets={'time': pn.widgets.Select})  # widget_location='top_left'
display(fig)

In [ ]:

# Use CSV into a color palette

#read in Forel RGB values
# csv = os.path.expanduser('~/data/WQ/FURGB.csv')
# fu_rgb = pd.read_csv(csv,header=0)
# fu_index=fu_rgb.index
# #fu_rgb.loc(index=['1'])
# reda=xr.DataArray(fu_rgb.values[:,1])
# greena=xr.DataArray(fu_rgb.values[:,2])
# bluea=xr.DataArray(fu_rgb.values[:,3])




In [ ]:
#index RGB values to Forel Integers with Xarray****needs work
# landsat_ds['red_r']=reda[landsat_ds.FUint].where(landsat_ds.FUint>0)
# landsat_ds['green_r']=greena[landsat_ds.FUint].where(landsat_ds.FUint>0)
# landsat_ds['blue_r']=bluea[landsat_ds.FUint].where(landsat_ds.FUint>0)


In [ ]:
#PLOT THE Forel Ule in RGB 

# ds=landsat_ds.where(landsat_ds.FUint!=0)
# #figure height
# #x=np.array(landsat_ds.sizes['time']/2)

# #Plot One Time Slice
# #DEAPlotting.rgb(ds=landsat_ds.where(landsat_ds.FUint!=0), bands=['red_r', 'green_r', 'blue_r'], index=5,  robust=True)

# #Plot them all in a badly contrained way
# ds[['red_r', 'green_r', 'blue_r']].to_array().plot.imshow(col='time', figsize=(20,6*np.ceil(landsat_ds.sizes['time']/2)),col_wrap=2, robust=True)



In [ ]:
#Create Float datasets (Forel Ule, Forel Ule RGB) to export to tiff

# FUrgb=xr.Dataset({'red_r':landsat_ds.red_r,'green_r':landsat_ds.green_r,'blue_r':landsat_ds.blue_r}, attrs={'crs': landsat_ds.crs})
# FUrgbfloat=FUrgb.astype(float)
FU=xr.Dataset({'FU':landsat_ds.FU}, attrs={'crs': landsat_ds.crs})


In [ ]:
#Export Forel Ule RGB from time slice - RGB
# ds=FUrgbfloat.isel(time=5)
# write_geotiff(filename='MenindeeFUrgb.tif', dataset=ds)

In [ ]:
#Export Forel Ule Index for time slice - data
from datacube.utils.cog import write_cog
ds=FU.isel(time=5)
display(ds)
write_cog(fname='MekongDelta.tif', geo_im=ds.FU.load())